# Data augmentation notebook
The aim of this notebook is to apply different sound degradations to the clean audio files in order to create a more diverse dataset for training and evaluation of speech enhancement models.
There will be 5 different types of degradations applied:
1. Reduced bandwidth (300-6000 Hz)
2. Analogic noise (pink noise) + electronic hum (50 Hz)
3. Dynamic range compression (simulating low-quality recording devices) + saturation
4. Codec low bit-rate compression (e.g., MP3 at 8 kbps)
5. Combination of all the previous degradations to create a very challenging degraded audio version.

## 1. Libraries

In [11]:
# Libraries used in this notebook
import os
import torchaudio
import pandas as pd
from tqdm import tqdm
from pesq import pesq
from pystoi import stoi
import soundfile as sf
import numpy as np
from scipy.signal import butter, lfilter
import librosa
from pydub import AudioSegment
import tempfile

# import the src folder
import sys
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))
try:
    from quality_scores import butter_lowpass_filter, compute_snr, compute_psnr, compute_mse, compute_pesq, compute_stoi
except Exception as e:
    print("Failed to import quality_scores from src:", e)
    raise

## 2. Data folders
Here we are going to use the high_quality_audio/ folder created in the previous notebook as our source of clean audio files. Each file will be processed to create multiple degraded versions, which will be saved in a new folder in the same data/librispeech_subset/ directory as follows:
```
data/
  └── librispeech_subset/
      ├── raw/
      ├── high_quality_audio/
      └── degraded_audio/
          ├── reduced_bandwidth/
          ├── analog_noise_hum/
          ├── dynamic_range_compression/
          ├── low_bitrate_codec/
          └── combined_degradations/
```
We will also create a new metadata CSV file that includes paths to the degraded audio files along with their corresponding PESQ, STOI, and SNR values.

In [12]:
DATA_DIR = "../data/librispeech_subset"
OG_DIR = os.path.join(DATA_DIR, "high_quality_audio")
os.makedirs(OG_DIR, exist_ok=True)
AUGMENTED_METADATA_CSV = os.path.join(DATA_DIR, "augmented_metadata.csv")
DEGRADED_DIR = os.path.join(DATA_DIR, "degraded_audio")
os.makedirs(DEGRADED_DIR, exist_ok=True)


## 3. Functions for degradations
Here we will define functions to apply each of the 5 degradations mentioned above.

In [13]:
# First degradation: Reduced bandwidth (300-6000 Hz)
def bandpass_filter(y, sr=16000, lowcut=300.0, highcut=6000.0):
    """Apply a bandpass filter to the audio data.
    Args:
        y (np.ndarray): Input audio signal.
        sr (int): Sampling rate of the audio signal.
        lowcut (float): Low cutoff frequency in Hz.
        highcut (float): High cutoff frequency in Hz.
    Uses : Butterworth bandpass filter implementation from scipy.signal library.
    Returns:
        np.ndarray: Filtered audio signal.
    
    """
    nyq = 0.5 * sr # Nyquist Frequency
    low = lowcut / nyq # Normalized Low Frequency
    high = highcut / nyq # Normalized High Frequency
    b, a = butter(6, [low, high], btype='band') # 6th order Butterworth bandpass filter
    y = lfilter(b, a, y)
    return y

In [14]:
# Second degradation: Analogic noise (pink noise) + electronic hum (50 Hz)
import numpy as np

def pink_noise(length):
    """
    Generate pink noise using the Voss-McCartney algorithm.
    Produces realistic analog tape hiss.
    """
    num_rows = 16  # more rows = smoother large-scale variations
    array = np.random.randn(num_rows, length)
    array = np.cumsum(array, axis=1)
    pink = np.sum(array, axis=0)
    pink = pink / np.max(np.abs(pink))  # normalize to [-1, 1]
    return pink

def add_pink_hiss(y, hiss_level_db=-25):
    """
    Add pink noise at a controlled level (in dB relative to signal).
    
    Args:
        y: audio signal (numpy array)
        hiss_level_db: level of added hiss (more negative = softer)
    """
    hiss = pink_noise(len(y))

    # Convert dB to amplitude ratio
    amp = 10 ** (hiss_level_db / 20.0)

    return y + hiss * amp

def add_pink_noise_and_hum(y, sr=16000, hiss_level_db=-25, hum_freq=50, hum_level_db=-30):
    """
    Add pink noise and a 50 Hz hum to the audio signal.
    
    Args:
        y: audio signal (numpy array)
        sr: sampling rate
        hiss_level_db: level of added hiss (more negative = softer)
        hum_freq: frequency of the hum (default 50 Hz)
        hum_level_db: level of added hum (more negative = softer)
    """
    # Add pink noise
    y_noisy = add_pink_hiss(y, hiss_level_db)

    # Generate hum signal
    t = np.arange(len(y)) / sr
    hum = np.sin(2*np.pi*hum_freq*t) + 0.5*np.sin(2*np.pi*2*hum_freq*t)
    hum = hum / np.max(np.abs(hum))  # normalization before mixing

    # Convert dB to amplitude ratio
    amp = 10 ** (hum_level_db / 20.0)

    # Add hum to the noisy signal
    y_noisy += hum * amp

    # To avoid clipping, normalize the final signal
    max_amp = np.max(np.abs(y_noisy))
    if max_amp > 1.0:
        y_noisy = y_noisy / max_amp

    return y_noisy


In [15]:
# Third degradation: Dynamic range compression (simulating low-quality recording devices) + saturation
import numpy as np

def envelope_rms(y, sr, attack=0.01, release=0.1):
    """
    Smooth RMS envelope detector.
    Args:
        y: audio signal (numpy array)
        sr: sampling rate
        attack: attack time in seconds
        release: release time in seconds
    Returns:
        np.ndarray: RMS envelope of the audio signal.
    """
    env = np.zeros_like(y)
    alpha_a = np.exp(-1/(attack*sr))
    alpha_r = np.exp(-1/(release*sr))

    for i in range(1, len(y)):
        rect = abs(y[i])
        if rect > env[i-1]:
            env[i] = alpha_a * env[i-1] + (1-alpha_a) * rect
        else:
            env[i] = alpha_r * env[i-1] + (1-alpha_r) * rect
    return env


def compressor(y, env, threshold=0.1, ratio=4.0):
    """
    Simple feed-forward compressor in linear scale.
    Args:
        y: audio signal (numpy array)
        env: RMS envelope of the audio signal, computed with envelope_rms().
        threshold: compression threshold, below which no compression is applied
        ratio: compression ratio, higher values = more compression
    Returns:
        np.ndarray: Compressed audio signal
    """
    gain = np.ones_like(y)

    above = env > threshold # indices where compression is applied
    gain[above] = threshold + (env[above] - threshold) / ratio
    gain[above] /= env[above]

    return y * gain


def saturation(y, drive=2.0):
    """
    Soft clipping with tanh to add analog warmth.
    Args:
        y: audio signal (numpy array)
        drive: amount of saturation (higher = more distortion)
    Returns:
        np.ndarray: Saturated audio signal.
    """
    y = np.tanh(y * drive)
    y = y / (np.max(np.abs(y)) + 1e-8)  # avoid clipping output
    return y


def degrade_compression_saturation(y, sr=16000,
                                   threshold=0.1, ratio=4.0,
                                   attack=0.01, release=0.1,
                                   drive=2.0):
    """
    Full effect: compression + saturation.
    Simulates cheap preamp / cassette recorder / old mic.
    Args:
        y: audio signal (numpy array)
        sr: sampling rate
        threshold: compression threshold, below which no compression is applied
        ratio: compression ratio, higher values = more compression
        attack: attack time in seconds
        release: release time in seconds
        drive: amount of saturation (higher = more distortion)
    Returns:
        np.ndarray: Degraded audio signal
    """
    env = envelope_rms(y, sr, attack, release)
    y_comp = compressor(y, env, threshold, ratio)
    y_sat = saturation(y_comp, drive)
    return y_sat


In [16]:
# Fourth degradation: Codec low bit-rate compression (e.g., MP3 at 8 kbps)
def save_temp_wav(y, sr):
    """Save audio to a temporary WAV file and return path.
    Args:
        y: audio signal (numpy array)
        sr: sampling rate
    Returns:
        str: Path to the temporary WAV file.
    """
    tmp_file = tempfile.NamedTemporaryFile(suffix=".wav", delete=False)
    sf.write(tmp_file.name, y, sr)
    return tmp_file.name

def encode_low_bitrate(input_wav, codec="mp3", bitrate="8k"):
    """Encode WAV to low bitrate codec (MP3 or Opus) and return path of output WAV.
    Args:
        input_wav: path to input WAV file
        codec: codec to use ("mp3" or "opus")
        bitrate: target bitrate (e.g., "8k" for 8 kbps)
    Returns:
        str: Path to the output WAV file.
    """
    audio = AudioSegment.from_wav(input_wav)
    output_wav = input_wav.replace(".wav", f"_{codec}.wav")
    audio.export(output_wav, format=codec, bitrate=bitrate)
    return output_wav

def load_wav(path, sr=16000):
    """Load WAV and return numpy array.
    Args:
        path: path to WAV file
        sr: target sampling rate
    Returns:
        np.ndarray: Loaded audio signal.
    """
    y, _ = librosa.load(path, sr=sr, mono=True)
    return y

def degrade_low_bitrate_codec(y, sr=16000, codec="mp3", bitrate="8k"):
    """
    Simulate low-bitrate codec compression.
    Returns degraded numpy array audio.
    Args:
        y: audio signal (numpy array)
        sr: sampling rate
        codec: codec to use ("mp3" or "opus")
        bitrate: target bitrate (e.g., "8k" for 8 kbps)
    Returns:
        np.ndarray: Degraded audio signal.
    """
    # 1. Save temporary WAV
    tmp_wav = save_temp_wav(y, sr)

    # 2. Encode low bitrate
    tmp_encoded = encode_low_bitrate(tmp_wav, codec, bitrate)

    # 3. Load encoded audio
    y_codec = load_wav(tmp_encoded, sr)

    # 4. Clean up temporary files
    os.remove(tmp_wav)
    os.remove(tmp_encoded)

    return y_codec

In [17]:
# Fifth degradation: Combination of all the previous degradations to create a very challenging degraded audio version. 
# Each degradation's parameters will be set to moderate levels to avoid complete loss of intelligibility.

def apply_all_degradations(y, sr=16000, hiss_level_db=-25, hum_freq=50, hum_level_db=-30, drive=2.0, codec="mp3", bitrate="8k"):
    y_noisy = add_pink_hiss(y, hiss_level_db)
    y_noisy = add_pink_noise_and_hum(y_noisy, sr, hiss_level_db, hum_freq, hum_level_db)
    y_noisy = degrade_compression_saturation(y_noisy, sr, drive=drive)
    y_noisy = degrade_low_bitrate_codec(y_noisy, sr, codec, bitrate)
    return y_noisy

## 4. Functions for saving audio files and updating metadata
Here we will define functions to save the degraded audio files and update the metadata CSV file accordingly.(using the previous metadata for the original clean files)

In [18]:
# Functions for saving audio files and updating metadata
def save_degraded_audio(y, sr, output_path):
    sf.write(output_path, y, sr)
    return output_path

def compute_metrics(y_clean, y_degraded, sr=16000):
    """Compute quality metrics comparing degraded audio to clean reference"""
    # Normalize both signals
    y_clean = y_clean / (np.max(np.abs(y_clean)) + 1e-8)
    y_degraded = y_degraded / (np.max(np.abs(y_degraded)) + 1e-8)
    
    # Ensure same length
    min_len = min(len(y_clean), len(y_degraded))
    y_clean = y_clean[:min_len]
    y_degraded = y_degraded[:min_len]
    
    # Calculate SNR on degraded signal
    frame_length = 2048
    hop_length = 512
    rms = librosa.feature.rms(y=y_degraded, frame_length=frame_length, hop_length=hop_length)
    noise_floor = np.percentile(rms, 10)
    signal_power = np.mean(y_degraded ** 2)
    # Compute the scores using the imported functions
    snr = compute_snr(y_clean, y_degraded)
    psnr = compute_psnr(y_clean, y_degraded)
    mse = compute_mse(y_clean, y_degraded)
    pesq_score = compute_pesq(y_clean, y_degraded, sr)
    stoi_score = compute_stoi(y_clean, y_degraded, sr)
    return snr, psnr, mse, pesq_score, stoi_score

def single_degradation_process(y_clean, original_path, degradation_type, degradation_params, output_dir, augmented_metadata):
    # Apply degradation
    if degradation_type == "reduced_bandwidth":
        y_degraded = bandpass_filter(y_clean)
    elif degradation_type == "analog_noise_hum":
        y_degraded = add_pink_noise_and_hum(y_clean, **degradation_params)
    elif degradation_type == "dynamic_range_compression":
        y_degraded = degrade_compression_saturation(y_clean, **degradation_params)
    elif degradation_type == "low_bitrate_codec":
        y_degraded = degrade_low_bitrate_codec(y_clean, **degradation_params)
    elif degradation_type == "combined_degradations":
        y_degraded = apply_all_degradations(y_clean, **degradation_params)
    else:
        raise ValueError("Unknown degradation type")

    # Save degraded audio
    base_filename = os.path.basename(original_path)
    degraded_path = os.path.join(output_dir, f"{base_filename}_{degradation_type}.wav")
    
    # Compute metrics comparing degraded to clean
    snr, psnr, mse, pesq_score, stoi_score = compute_metrics(y_clean, y_degraded, sr=16000)
    
    # Update metadata
    augmented_metadata.append({
        "file_path": degraded_path,
        "original_file_path": original_path,
        "degradation": degradation_type,
        "snr": snr,
        "psnr": psnr,
        "mse": mse,
        "pesq": pesq_score,
        "stoi": stoi_score,
        "duration_s": len(y_degraded) / 16000
    })

    # Save degraded audio
    save_degraded_audio(y_degraded, 16000, degraded_path)

    return y_degraded, degraded_path

In [19]:
# Create augmented metadata CSV
metadata_csv = "../data/librispeech_subset/metadata.csv"
metadata = pd.read_csv(metadata_csv)

# Create directory for augmented files
augmented_dir = "../data/librispeech_subset/degraded_audio"
os.makedirs(augmented_dir, exist_ok=True)
first_degradation_dir = os.path.join(augmented_dir, "reduced_bandwidth")
os.makedirs(first_degradation_dir, exist_ok=True)
second_degradation_dir = os.path.join(augmented_dir, "analog_noise_hum")
os.makedirs(second_degradation_dir, exist_ok=True)
third_degradation_dir = os.path.join(augmented_dir, "dynamic_range_compression")
os.makedirs(third_degradation_dir, exist_ok=True)
fourth_degradation_dir = os.path.join(augmented_dir, "low_bitrate_codec")
os.makedirs(fourth_degradation_dir, exist_ok=True)
fifth_degradation_dir = os.path.join(augmented_dir, "combined_degradations")
os.makedirs(fifth_degradation_dir, exist_ok=True)



In [21]:
# Unlike metadata, augmented metadata will also contain the path of the original clean files
# As well as the degradation parameters used
augmented_metadata = []


for index, row in tqdm(metadata.iterrows(), total=len(metadata)):
    # First, write the metadata for the clean file in the augmented metadata
    augmented_metadata.append({
        "file_path": os.path.join(OG_DIR , row["file"]),
        "original_file_path": os.path.join(OG_DIR , row["file"]),
        "degradation": "clean",
        "snr": row["snr"],
        "psnr": row["psnr"],
        "mse": row["mse"],
        "pesq": row["pesq"],
        "stoi": row["stoi"],
        "duration_s": row["duration_s"]
    })
    # Load clean audio : OG_DIR + row["file"]
    print(row["file"])
    clean_path = os.path.join(OG_DIR , row["file"])
    y_clean, sr = librosa.load(clean_path, sr=16000, mono=True)
    # Apply degradations
    # 1. Reduced bandwidth
    y_degraded, degraded_path = single_degradation_process(y_clean, clean_path, "reduced_bandwidth", {}, first_degradation_dir, augmented_metadata)
    # 2. Analog noise and hum
    degradation_params = {"hiss_level_db": -25, "hum_freq": 50, "hum_level_db": -30}
    y_degraded, degraded_path = single_degradation_process(y_clean, clean_path, "analog_noise_hum", degradation_params, second_degradation_dir, augmented_metadata)
    # 3. Dynamic range compression + saturation
    degradation_params = {"threshold": 0.1, "ratio": 4.0, "attack": 0.01, "release": 0.1, "drive": 2.0}
    y_degraded, degraded_path = single_degradation_process(y_clean, clean_path, "dynamic_range_compression", degradation_params, third_degradation_dir, augmented_metadata)
    # 4. Low bitrate codec
    degradation_params = {"codec": "mp3", "bitrate": "8k"}
    y_degraded, degraded_path = single_degradation_process(y_clean, clean_path, "low_bitrate_codec", degradation_params, fourth_degradation_dir, augmented_metadata)
    # 5. Combined degradations
    degradation_params = {"hiss_level_db": -25, "hum_freq": 50, "hum_level_db": -30, "drive": 2.0, "codec": "mp3", "bitrate": "8k"}
    y_degraded, degraded_path = single_degradation_process(y_clean, clean_path, "combined_degradations", degradation_params, fifth_degradation_dir, augmented_metadata)

# Save augmented metadata
augmented_metadata_df = pd.DataFrame(augmented_metadata)
augmented_metadata_df.to_csv("../data/librispeech_subset/augmented_metadata.csv", index=False)
    
    
    
    

  0%|          | 0/293 [00:00<?, ?it/s]

librispeech_sample_0003.wav


  0%|          | 1/293 [00:03<17:55,  3.68s/it]

librispeech_sample_0005.wav


  1%|          | 2/293 [00:05<13:08,  2.71s/it]

...

 99%|█████████▉| 291/293 [08:11<00:03,  1.84s/it]

librispeech_sample_2669.wav


100%|█████████▉| 292/293 [08:12<00:01,  1.69s/it]

librispeech_sample_2692.wav


100%|██████████| 293/293 [08:14<00:00,  1.69s/it]
